In [1]:
# ドライブ読み込み
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks"

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [2]:
!pip install japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=e77af27a0899c77684fa70d9646985ddfae8b56fdcb236e2e1f2af9444eee531
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import sympy
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import sklearn
from sklearn.metrics import mean_absolute_error

# データ読み込み

In [4]:
# --------------------(1)-----------------------
import pandas as pd
columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF', 'SalePrice']
df = pd.read_csv('AmesHousing.tsv', sep='\t', usecols=columns)
print(df)

      Overall Qual  Overall Cond  Total Bsmt SF Central Air  Gr Liv Area  \
0                6             5         1080.0           Y         1656   
1                5             6          882.0           Y          896   
2                6             6         1329.0           Y         1329   
3                7             5         2110.0           Y         2110   
4                5             5          928.0           Y         1629   
...            ...           ...            ...         ...          ...   
2925             6             6         1003.0           Y         1003   
2926             5             5          864.0           Y          902   
2927             5             5          912.0           Y          970   
2928             5             5         1389.0           Y         1389   
2929             7             5          996.0           Y         2000   

      SalePrice  
0        215000  
1        105000  
2        172000  
3        244000

# 辞書適応

In [5]:
# --------------------(2)-----------------------
# 数値でない変数を数値化
df['Central Air'] = df['Central Air'].map({'N':0, 'Y':1})
# print(df.head())

#欠損値の確認

In [6]:
# --------------------(3)-----------------------
print(df.isnull().sum())  # 欠損値の確認。
print()
df2 = df.dropna(axis=0)    # 欠損値のある行は削除。

Overall Qual     0
Overall Cond     0
Total Bsmt SF    1
Central Air      0
Gr Liv Area      0
SalePrice        0
dtype: int64



# データ分割

In [7]:
# --------------------(4)-----------------------
a_train, a_test = train_test_split(df2, train_size=0.8, random_state=0)

# 重回帰分析

In [8]:
# --------------------(5)-----------------------
# 説明変数と目的変数を分ける
X = a_train[['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF']]
y = a_train['SalePrice']
# モデルのfit
model = LinearRegression()
model.fit(X, y)
print('学習データを重回帰分析してできた近似式は以下の通り')
print(f'SalePrice = {model.coef_[0]} * “Overall Qual”+ {model.coef_[1]} * “Overall Cond"')
print(f'+ {model.coef_[2]} * “Gr Liv Area”+ {model.coef_[3]} * “Central Air"')
print(f' + {model.coef_[4]} * "Total Bsmt SF" + {model.intercept_}')

学習データを重回帰分析してできた近似式は以下の通り
SalePrice = 27634.80886749324 * “Overall Qual”+ 868.020228609245 * “Overall Cond"
+ 51.815272800037974 * “Gr Liv Area”+ 12875.167372339052 * “Central Air"
 + 36.75953841768876 * "Total Bsmt SF" + -121195.26532838467


# ラッソ回帰分析

In [9]:
# --------------------(6)-----------------------
modelL = Lasso(alpha=10, max_iter=2000)
modelL.fit(X, y)

print('ラッソ回帰分析してできた近似式は以下の通り')
print(f'SalePrice = {modelL.coef_[0]} * “Overall Qual”+ {modelL.coef_[1]} * “Overall Cond"')
print(f'+ {modelL.coef_[2]} * “Gr Liv Area”+ {modelL.coef_[3]} * “Central Air"')
print(f' + {modelL.coef_[4]} * "Total Bsmt SF" + {modelL.intercept_}')

ラッソ回帰分析してできた近似式は以下の通り
SalePrice = 27634.444617247977 * “Overall Qual”+ 866.0811259701828 * “Overall Cond"
+ 51.818086193094246 * “Gr Liv Area”+ 12709.143307923126 * “Central Air"
 + 36.77806620829118 * "Total Bsmt SF" + -121051.10541990024


# リッジ回帰分析

In [10]:
# --------------------(7)-----------------------
modelR = Ridge(alpha=10, max_iter=2000)
modelR.fit(X, y)

print('リッジ回帰分析してできた近似式は以下の通り')
print(f'SalePrice = {modelR.coef_[0]} * “Overall Qual”+ {modelR.coef_[1]} * “Overall Cond"')
print(f'+ {modelR.coef_[2]} * “Gr Liv Area”+ {modelR.coef_[3]} * “Central Air"')
print(f' + {modelR.coef_[4]} * "Total Bsmt SF" + {modelR.intercept_}')

リッジ回帰分析してできた近似式は以下の通り
SalePrice = 27573.020563500024 * “Overall Qual”+ 898.8163756322127 * “Overall Cond"
+ 51.90131438878146 * “Gr Liv Area”+ 12046.974196373667 * “Central Air"
 + 36.93842124769142 * "Total Bsmt SF" + -120534.00433000838


各モデルに対してテストデータの予測値を計算

真の値と予測値の平均絶対値誤差の計算

In [11]:
# --------------------(8)-----------------------
# 説明変数と目的変数を分ける
X_test = a_test[['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF']]
y_test = a_test['SalePrice']

# 予測値の算出
y_pred = model.predict(X_test)
y_pred_R = modelR.predict(X_test)
y_pred_L = modelL.predict(X_test)

# 平均絶対値誤差の算出
mae = mean_absolute_error(y_test, y_pred)
mae_R = mean_absolute_error(y_test, y_pred_R)
mae_L = mean_absolute_error(y_test, y_pred_L)

# 出力
print('線形回帰の平均絶対値誤差：', mae)
print('リッジ回帰の平均絶対値誤差：', mae_R)
print('ラッソ回帰の平均絶対値誤差：', mae_L)

線形回帰の平均絶対値誤差： 26024.79610410499
リッジ回帰の平均絶対値誤差： 26003.170474850423
ラッソ回帰の平均絶対値誤差： 26022.863818837923
